In [6]:
import os
import json
import plotly.io
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import pandas as pd
import torch


def get_bt(FL, BD):
    while True:
        k_sample = np.random.normal(loc=0.1, scale=0.001)
        relative_k = (k_sample - 0.1) / 0.1
        if -0.1 <= relative_k <= 0.1:
            break
    return FL * BD * k_sample

def von_bertalanffy(t, l_inf=85, l_0=4.0, k=0.1):
    return l_inf - (l_inf - l_0) * np.exp(-k * t)

def generate_random_curves(n, t, l_inf_range=(84, 86), l_0_range=(3.8, 4.2), k_range=(0.095, 0.105)):
    fl_curves = []
    for _ in range(n):
        l_inf = np.random.uniform(*l_inf_range)
        l_0 = np.random.uniform(*l_0_range)
        k = np.random.uniform(*k_range)
        fl_curve = von_bertalanffy(t, l_inf, l_0, k)
        fl_curves.append(fl_curve)
    return np.array(fl_curves)

def generate_noise_curve(mean_curve, noise_scale=1.0, relative_noise_limit=0.1, incremental_noise=0.3):
    noise_curve = np.zeros_like(mean_curve)
    
    for i in range(len(mean_curve)):
        while True:
            if i == 0:
                noise_curve[i] = mean_curve[i]
                break
            
            noise_sample = np.random.normal(loc=mean_curve[i], scale=noise_scale)
            relative_noise = (noise_sample - mean_curve[i]) / mean_curve[i]
            
            if -relative_noise_limit <= relative_noise <= relative_noise_limit:
                if noise_sample > noise_curve[i-1]:
                    noise_curve[i] = noise_sample
                    break
                else:
                    noise_curve[i] = noise_curve[i-1] + np.random.uniform(0, incremental_noise)
                    break
    return noise_curve

def generate_predicted_weight(FL, BD, k=0.001, noise_scale=0.001):
    bt = get_bt(FL, BD)
    predicted_weight = FL * BD * np.pi * k * bt
    noise_weights = np.zeros_like(predicted_weight)

    for i in range(len(predicted_weight)):
        while True:
            if i == 0:
                noise_weights[i] = predicted_weight[i]
                break
            
            noise_sample = np.random.normal(loc=predicted_weight[i], scale=noise_scale)
            relative_noise = (noise_sample - predicted_weight[i]) / predicted_weight[i]
            
            if -0.1 <= relative_noise <= 0.1:
                if noise_sample > noise_weights[i-1]:
                    noise_weights[i] = noise_sample
                    break
                else:
                    noise_weights[i] = noise_weights[i-1] + np.random.uniform(0, 0.3)
                    break
    return predicted_weight, noise_weights

class ModelConfig:
    def __init__(self):
        self.l_inf_range = (70, 90)
        self.l_0_range = (5, 7)
        self.k_range = (0.5, 0.6)
        
        self.fl_noise_config = {
            "noise_scale": 0.01, 
            "relative_noise_limit": 0.1, 
            "incremental_noise": 0.01
        }
        
        self.bd_noise_config = {
            "noise_scale": 0.001, 
            "relative_noise_limit": 0.1, 
            "incremental_noise": 0.001
        }
        
        self.weight_noise_scale = 0.1
        self.trail_num = 0
        self.training_count = 0
        
    def save_to_file(self, filename):
        with open(filename, 'w') as f:
            f.write(str(self.__dict__))
    
    def load_from_file(self, filename):
        with open(filename, 'r') as f:
            data = eval(f.read())
            self.__dict__.update(data)
            
def generate_data_and_dataframe(config):
    time_days = np.linspace(0, 1922, 1922) + 600
    time_years = time_days / 365  

    fl_curves = generate_random_curves(10, time_years, config.l_inf_range, config.l_0_range, config.k_range)
    mean_fl_curve = np.mean(fl_curves, axis=0)
    fl_noise = generate_noise_curve(mean_fl_curve, **config.fl_noise_config)

    bd_curves = generate_random_curves(10, time_years, config.l_inf_range, config.l_0_range, config.k_range)
    bd_curves = bd_curves * 0.169599
    mean_bd_curve = np.mean(bd_curves, axis=0)
    bd_noise = generate_noise_curve(mean_bd_curve, **config.bd_noise_config)

    result_weights, noise_weight = generate_predicted_weight(fl_noise, bd_noise, noise_scale=config.weight_noise_scale)

    data = {
        "Time_Days": time_days[:-2],
        "Time_Years": time_years[:-2],
        "FL_Noise": fl_noise[:-2],
        "BD_Noise": bd_noise[:-2],
        "Result_Weights": result_weights[:-2],
        "Noise_Weight": noise_weight[:-2],
        "Mean_FL_Curve": mean_fl_curve[:-2],  # 원래 곡선 추가
        "Mean_BD_Curve": mean_bd_curve[:-2]   # 원래 곡선 추가
    }
    df = pd.DataFrame(data)
    df['time_index'] = df.index
    df['group'] = 1
    
    return df

def generate_plot(config, df, save_path):
    dot_size = 2

    fig = make_subplots(rows=3, cols=1, subplot_titles=('FL Growth Curve', 'BD Growth Curve', 'Weight Growth Curve'))
    
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["Mean_FL_Curve"], mode='lines', name='Original FL Growth Curve', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["FL_Noise"], mode='lines + markers', name='FL Growth Curve with Noise', marker=dict(color='red', size=dot_size)), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["Mean_BD_Curve"], mode='lines', name='Original BD Growth Curve', line=dict(color='blue')), row=2, col=1)
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["BD_Noise"], mode='lines + markers', name='BD Growth Curve with Noise', marker=dict(color='red', size=dot_size)), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["Result_Weights"], mode='lines', name='Weight Growth Curve with Noise', marker=dict(color='blue')), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["Time_Days"], y=df["Noise_Weight"], mode='markers', name='Weight Growth Curve with Noise', marker=dict(color='red', size=dot_size)), row=3, col=1)
    
    fig.update_layout(title='Growth Curves with Noise', height=1200, width=1000)

    # Create directories if not exist and save the plot as HTML
    os.makedirs(save_path, exist_ok=True)
    plotly.io.write_html(fig, os.path.join(save_path, 'growth_curves_with_noise.html'))


In [12]:
text_path = 'config.txt'
config = ModelConfig()
config.load_from_file(text_path)

# You will need to define or import the generate_data_and_dataframe function
data = generate_data_and_dataframe(config)  
save_path = f"results"
generate_plot(config, data, save_path)

#data = data[['FL_Noise' , 'BD_Noise' , 'Noise_Weight' , 'time_index' , 'group']]
data

,Time_Days,Time_Years,FL_Noise,BD_Noise,Result_Weights,Noise_Weight,Mean_FL_Curve,Mean_BD_Curve,time_index,group
0,600.000000,1.643836,49.422842,8.956323,61.161149,61.161149,49.422842,8.956323,0,1
1,601.000521,1.646577,49.458497,8.966914,61.394371,61.678837,49.466391,8.964490,1,1
2,602.001041,1.649318,49.584611,8.969682,61.745969,61.795209,49.509872,8.972645,2,1
3,603.001562,1.652059,49.593701,8.978005,61.883303,62.078050,49.553285,8.980787,3,1
4,604.002082,1.654800,49.594099,8.989853,62.047730,62.253578,49.596630,8.988917,4,1
...,...,...,...,...,...,...,...,...,...,...
1915,2515.996877,6.893142,76.714955,14.061476,363.230799,418.848367,75.883224,14.051704,1915,1
1916,2516.997397,6.895883,76.723893,14.061546,363.319057,419.065781,75.885405,14.052154,1916,1
1917,2517.997918,6.898624,76.731203,14.061624,363.392325,419.363489,75.887582,14.052603,1917,1
1918,2518.998438,6.901366,76.736656,14.061815,363.453828,419.525057,75.889757,14.053052,1918,1


In [13]:
generate_plot(config, data, './')

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# Calculate NRMSE
def calculate_nrmse(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)
    nrmse = rmse / (y_true.max() - y_true.min())
    return nrmse

# LightGBM model training and evaluation function
def light_gbm_nrmse(data, hparams, test_size=0.2, random_state=1):
    # Extracting features and target variable from the dataframe
    y = data['Noise_Weight']
    X = data[['FL_Noise', 'BD_Noise']]

    print("Target Shape:", y.shape)
    print("Features Shape:", X.shape)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Train the LightGBM model
    lgb_model = lgb.LGBMRegressor(**hparams)
    lgb_model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric='l1',
                  early_stopping_rounds=1000)

    # Predict the target variable for the test set
    y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration_)

    # Calculate NRMSE for the model
    nrmse = calculate_nrmse(y_test, y_pred)

    result_df = pd.DataFrame({'Model': ['LightGBM'], 'NRMSE': [nrmse]})
    return result_df

# Hyperparameters for LightGBM (You might want to fine-tune these)
hparams = {
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100
}

# Run the LightGBM model training and evaluation
result_df = light_gbm_nrmse(data, hparams)
print(result_df)


Target Shape: (1920,)
Features Shape: (1920, 2)
[1]	valid_0's l1: 82.1224	valid_0's l2: 9066.36
[2]	valid_0's l1: 78.018	valid_0's l2: 8183.51
[3]	valid_0's l1: 74.1092	valid_0's l2: 7384.49
[4]	valid_0's l1: 70.3985	valid_0's l2: 6664.93
[5]	valid_0's l1: 66.8859	valid_0's l2: 6016.03
[6]	valid_0's l1: 63.5425	valid_0's l2: 5429.99
[7]	valid_0's l1: 60.3737	valid_0's l2: 4901.59
[8]	valid_0's l1: 57.3636	valid_0's l2: 4424.5
[9]	valid_0's l1: 54.5002	valid_0's l2: 3993.54
[10]	valid_0's l1: 51.7778	valid_0's l2: 3604.78
[11]	valid_0's l1: 49.188	valid_0's l2: 3253.7
[12]	valid_0's l1: 46.7257	valid_0's l2: 2936.37
[13]	valid_0's l1: 44.393	valid_0's l2: 2650.22
[14]	valid_0's l1: 42.18	valid_0's l2: 2392.47
[15]	valid_0's l1: 40.0706	valid_0's l2: 2159.67
[16]	valid_0's l1: 38.0689	valid_0's l2: 1949.19
[17]	valid_0's l1: 36.1703	valid_0's l2: 1759.75
[18]	valid_0's l1: 34.3654	valid_0's l2: 1588.51
[19]	valid_0's l1: 32.6569	valid_0's l2: 1434.36
[20]	valid_0's l1: 31.0222	valid_0's 

/home/nplab/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning:

'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.

